## Ray casting алгоритм для множества точек и множества полигонов

In [38]:
import sys
import pandas as pd

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Polygon:
    def __init__(self, points):
        self.points = points
        
        self.min_x = min(points, key=lambda p: p.x).x
        self.max_x = max(points, key=lambda p: p.x).x
        self.min_y = min(points, key=lambda p: p.y).y
        self.max_y = max(points, key=lambda p: p.y).y
    
    @property
    def edges(self):
        edge_list = []
        for i,p in enumerate(self.points):
            p1 = p
            p2 = self.points[(i+1) % len(self.points)]
            edge_list.append((p1,p2))

        return edge_list
    
    def contains(self, point):
        _huge = sys.float_info.max
        _eps = 0.00001

        inside = False
        for edge in self.edges: 
            A, B = edge[0], edge[1]
            if A.y > B.y:
                A, B = B, A

            if point.y == A.y or point.y == B.y:
                point.y += _eps

            if (point.y > B.y or point.y < A.y or point.x > max(A.x, B.x)):
                continue

            if point.x < min(A.x, B.x):
                inside = not inside
                continue

            try:
                m_edge = (B.y - A.y) / (B.x - A.x)
            except ZeroDivisionError:
                m_edge = _huge

            try:
                m_point = (point.y - A.y) / (point.x - A.x)
            except ZeroDivisionError:
                m_point = _huge

            if m_point >= m_edge:
                inside = not inside
                continue

        return inside

In [39]:
def get_points(points_table):
    user_coords = points_table.itertuples(name=None, index=False)
    points = {}

    for p in user_coords:
        points[p[0]] = Point(p[1], p[2])
        
    return points

def get_polygons(polygon_table):
    polygons_coords = polygon_table.itertuples(name=None, index=False)
    
    polygons = {}
    current_id = None
    current_points = []
    
    for p in polygons_coords:
        poly_id = p[0]
        
        if current_id == poly_id:
            current_points.append(Point(p[1], p[2])) 
        else:
            if current_points:
                polygons[current_id] = Polygon(current_points)
                current_points = []
            current_points.append(Point(p[1], p[2]))
            current_id = poly_id
            
            
    polygons[current_id] = Polygon(current_points)
            
    return polygons

def count_points_entries(user_coords, polygons_coords):
    points = get_points(user_coords).items()
    polygons = get_polygons(polygons_coords).items()
    entries = {}
    
    for point_id, point in points:
        entries[point_id] = 0
        
        filtered_polygons = filter(lambda poly: compare_coords(point, poly[1]), polygons)
            
        for poly_id, polygon in filtered_polygons:
            is_inside = polygon.contains(point)
            if is_inside:
                entries[point_id] += 1
                        
    return entries

def compare_coords(point, poly):
    if point.x < poly.min_x or point.x > poly.max_x or point.y < poly.min_y or point.y > poly.max_y:
        return False
    
    return True

def create_entries_table(user_coords, polygons):
    entries = count_points_entries(user_coords, polygons)
    return pd.DataFrame(entries.items(), columns=['id', 'number_of_places_available'])
    

## Тест

In [40]:
user_coordinates = pd.read_csv('user_coordinates.csv')
place_zone_coordinates = pd.read_csv('place_zone_coordinates.csv')

In [41]:
result = create_entries_table(user_coordinates, place_zone_coordinates)
result.sort_values(by='number_of_places_available', ascending=False)

,id,number_of_places_available
384,384,5
80,80,4
730,730,4
451,451,4
298,298,4
...,...,...
455,455,0
456,456,0
458,458,0
459,459,0
